In [37]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPEN_AI_KEY")

Get video transcript or audio file if video has no captions

In [38]:
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from datetime import date

videos = open("inputs.txt", "r")
video_obj = {}

for video in videos:
    yt = YouTube(video)
    video_obj[yt.title] = { "author": yt.author, "transcribed_on": date.today() }
    caption=YouTubeTranscriptApi.get_transcript(yt.video_id, languages=['en'])
    if not caption:
        streams = yt.streams.filter(only_audio=True, file_extension="mp4")
        streams[-1].download("audio_files")
    else:
        f = open("transcripts/" + yt.title + ".txt", "w")
        for obj in caption:
            f.write(obj['text'])
        f.close()

Transcribe audio files (if they exist)

In [39]:
import subprocess
import sys

audio_files = os.fsencode("audio_files")

for file in os.listdir(audio_files):
    filename = os.fsdecode(file)
    if filename.endswith(".mp4"):
        title = filename.replace(".mp4", "")

        # convert mp4 file to wav (16-bit)
        new_filename = title + ".wav"
        subprocess.run([
            'ffmpeg',
            '-i', os.path.join("audio_files", filename),
            '-ar', '16000',
            '-ac', '1',
            '-c:a', 'pcm_s16le',
            os.path.join("audio_files", new_filename)
        ])

        # call local whisper model
        result = subprocess.check_output([
            './main',
            '-f', '../Podcast-Summarizer/audio_files/' + new_filename
        ], cwd="../whisper.cpp")

        f = open("transcripts/" + title + ".txt", "w")
        f.write(result.decode(sys.stdout.encoding).strip())
        f.close()

print(video_obj)


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopen

{'I Survived 7 Days In An Abandoned City': {'author': 'MrBeast', 'transcribed_on': datetime.date(2024, 3, 10)}}



whisper_print_timings:     load time =   129.78 ms
whisper_print_timings:     fallbacks =   0 p /   0 h
whisper_print_timings:      mel time =   512.66 ms
whisper_print_timings:   sample time =  8592.47 ms / 28249 runs (    0.30 ms per run)
whisper_print_timings:   encode time =  2948.23 ms /    37 runs (   79.68 ms per run)
whisper_print_timings:   decode time =    69.19 ms /    16 runs (    4.32 ms per run)
whisper_print_timings:   batchd time = 32341.54 ms / 28049 runs (    1.15 ms per run)
whisper_print_timings:   prompt time =   695.83 ms /  8055 runs (    0.09 ms per run)
whisper_print_timings:    total time = 45346.41 ms
ggml_metal_free: deallocating
ggml_metal_free: deallocating
